In [3]:
%reload_ext autoreload
%autoreload 2
import sys, os
print(sys.executable)
module_path = os.path.abspath(os.path.join('..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)
# Verify kernal path
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


True

In [4]:
import src.config as config
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
2024-08-08 01:43:35,185 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-08-08 01:43:35,186 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# Feature view is defined to read data from feature store, define how you want to fetch data from potentially many different feature groups and merge it to get the final dataset. 
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feaeture view already existed. Skip creation")

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feaeture view already existed. Skip creation


In [6]:
# feature-view-obj.func(): gets training data-frame from hopsworks
ts_prices, _ = feature_view.training_data(
    description="Time-series daily stock prices"
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 
2024-08-08 01:44:01,002 WARNING: VersionWarning: Incremented version to `36`.



In [7]:
# sort the data fetched from feature-store and make sure it is sorted, the last row should be the current date because we are getting most recetn data with feature pipeline
ts_prices.sort_values(by=["datetime"], inplace=True)
ts_prices

,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,sma_5,sma_20,ema_5,ema_20,vol_5,daily_return,price_diff,volume_sma_5,volume_change
341,6.036234e+07,165.893900,166.370000,164.87,167.810000,164.200000,540017.0,2022-08-02 00:00:00+00:00,164.870,168.9830,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
691,6.036234e+07,165.893900,166.370000,164.87,167.810000,164.200000,540017.0,2022-08-03 00:00:00+00:00,164.870,168.9830,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
363,6.036234e+07,165.893900,166.370000,164.87,167.810000,164.200000,540017.0,2022-08-04 00:00:00+00:00,164.870,168.9830,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
698,6.036234e+07,165.893900,166.370000,164.87,167.810000,164.200000,540017.0,2022-08-05 00:00:00+00:00,164.870,168.9830,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
352,6.036234e+07,165.893900,166.370000,164.87,167.810000,164.200000,540017.0,2022-08-06 00:00:00+00:00,165.754,169.6270,164.870000,164.870000,1.948854,0.000000,0.00,62866611.4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,1.138001e+08,212.517967,205.776667,212.80,217.533333,203.236667,1485155.0,2024-08-04 00:00:00+00:00,217.886,221.3045,216.574198,220.029169,3.535517,-0.016318,-3.53,86840273.4,0.051585
439,1.026353e+08,221.951900,219.150000,219.86,225.600000,217.710000,1132479.0,2024-08-05 00:00:00+00:00,219.560,221.0860,219.761317,220.896606,0.670820,0.000000,0.00,94333305.4,0.000000
736,1.193825e+08,207.801000,199.090000,209.27,213.500000,196.000000,1661493.0,2024-08-05 04:00:00+00:00,215.324,220.0270,214.139466,219.004486,4.296106,-0.016588,-3.53,101032180.2,0.049054
735,1.193825e+08,207.801000,199.090000,209.27,213.500000,196.000000,1661493.0,2024-08-06 00:00:00+00:00,213.506,219.0465,212.516310,218.077392,4.602557,0.000000,0.00,112683633.2,0.000000


In [8]:
from src.data import transform_ts_data_into_features_target

n_previous_days = 12
step_size = 1
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size)
X = features
Y = targets
print(f'Features shape: {features.shape}')  # Should be (number_of_examples, n_previous_days, n_features)
print(f'Targets shape: {targets.shape}')    # Should be (number_of_examples,)


Number of sequences/examples: 724
Last val: 172.82666666666665
Target val: 173.19
Features shape: (724, 13, 16)
Targets shape: (724,)


In [1]:
# from src.data import train_test_split_tabular

# # combine features/targets to create tabular data
# features_and_target = pd.DataFrame(features.copy())
# features_and_target["target_close_price_next_day"] = targets   # target column name
# print(f"{features_and_target.shape=}")
# features_and_target

In [2]:
# from sklearn.model_selection import train_test_split
# from src.data import train_test_split_tabular

# X_train, X_test, y_train, y_test = train_test_split_tabular(features_and_target)
# print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
# print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
# print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
# print(f"y_test: {y_test.shape}")

In [9]:
# LIGHTGBM REGRESSOR MODEL
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


num_samples, sequence_length, num_features = X.shape
X_flattened = X.reshape(num_samples, sequence_length * num_features)

# Create the LightGBM Regressor model
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

# Train the model
lgb_model.fit(X_flattened, Y)

# Predict on the test set
y_pred = lgb_model.predict(X_flattened)

# Calculate the mean squared error
mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")

# Print the first few predictions and actual values
print("Predictions:", y_pred[:5])
print("Actual values:", Y[:5])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50257
[LightGBM] [Info] Number of data points in the train set: 724, number of used features: 208
[LightGBM] [Info] Start training from score 173.517983
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [31]:
# SAVE MODEL AS PICKLE LOCALLY
import joblib

model_path = f'../models/lgb_model3.pkl'
joblib.dump(lgb_model, model_path)


['../models/lgb_model3.pkl']

In [32]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# before saving model in hopsworks need to define its schema
input_schema = Schema(X)
output_schema = Schema(Y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)


In [30]:
# Connect to the Hopsworks model registry
model_registry = project.get_model_registry()

# Register the model
model = model_registry.sklearn.create_model(
    name="lightgbm_model_stock_prices3",
    metrics={"MSE": mse},
    description="LightGBM model for predicting stock prices 3",
    input_example=X[0].reshape(1, -1),
    model_schema=model_schema
)

# Upload the model file
model.save(model_path)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/269567 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3429 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/216 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/835736/models/lightgbm_model_stock_prices3/1


Model(name: 'lightgbm_model_stock_prices3', version: 1)